# Research Question

## Can the outcome(or the odds) of a football match be accurately found using prior match history, and match context.

- Will league affect certain teams?
- Will home team have the advantage?
- Do coaches affect win rate?

# The data

The data in this report contains match history for over 100k games, between teams in many different leagues. The columns we have are the following:
- Winning team(home or away)(binary)
- Home team(categorical)
- Away team(categorical)
- League name(categorical)
- Cup game(Binary)
- Home coach(categorical)
- Away coach(categorical)

Additionally, there are several columns with historical data, up to 10 matches prior, for both the home and away team for each match. Initially, we will not be using this data, but that may change



# Data Handling


In [5]:
import numpy as np
import pandas as pd
train=pd.read_csv(r"football-match-probability-prediction/train.csv")

test=pd.read_csv(r"football-match-probability-prediction/test.csv")

train_target_and_scores=pd.read_csv(r"football-match-probability-prediction/train_target_and_scores.csv")


(110938, 190)

In [2]:
train=train.iloc[:,:10]
train.head()


,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id
0,11906497,away,Newell's Old Boys,River Plate,2019-12-01 00:45:00,Superliga,636,False,468196.0,468200.0
1,11984383,home,Real Estelí,Deportivo Las Sabanas,2019-12-01 01:00:00,Primera Division,752,False,516788.0,22169161.0
2,11983301,draw,UPNFM,Marathón,2019-12-01 01:00:00,Liga Nacional,734,False,2510608.0,456313.0
3,11983471,away,León,Morelia,2019-12-01 01:00:00,Liga MX,743,False,1552508.0,465797.0
4,11883005,home,Cobán Imperial,Iztapa,2019-12-01 01:00:00,Liga Nacional,705,False,429958.0,426870.0


There is not much data handling to do, as most of our data is categorical, and already labelled properly with no missing values. Thus, we can begin smoothly with model building